# Explore results

In this notebook, I explore the similarity results in the search for COVID-19 candidates based on existing literature data.

In [2]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import numpy as np
import collections
import h5py
from chemicalchecker.util.plot.diagnosticsplot import set_style
set_style()

## Literature data

This is how the literature data looks like once pulled from the Google Spreadsheet and processed.

In [49]:
df_lit = pickle.load(open("../outputs/df_lit_cc.pkl", "rb"))
df_lit

,inchikey,name,evidence,moa,descriptions,links
0,KBOPZPXVLCULAV-UHFFFAOYSA-N,Mesalazine,0,5,"None, it´s a computational repurposing strateg...",https://www.nature.com/articles/s41421-020-0153-3
1,XFCLJVABOIYOMF-QPLCGJKRSA-N,Toremifene;Toremifene citrate,1,2,"None, it´s a computational repurposing strateg...",https://www.nature.com/articles/s41421-020-015...
2,JUKPWJGBANNWMW-VWBFHTRKSA-N,Eplerenone;Eplerenone,1,5,"None, it´s a computational repurposing strateg...",https://www.nature.com/articles/s41421-020-015...
3,AHOUBRCZNHFOSL-YOEHRIQHSA-N,Paroxetine,0,0,"None, it´s a computational repurposing strateg...",https://www.nature.com/articles/s41421-020-0153-3
4,QFJCIRLUMZQUOT-HPLJOQBZSA-N,Sirolimus;Sirolimus;Rapamycin;Rapamycin,2,5,"None, it´s a computational repurposing strateg...",https://www.nature.com/articles/s41421-020-015...
...,...,...,...,...,...,...
146,FAKRSMQSSFJEIM-RQJHMYQMSA-N,Captopril,0,-2,Drugs from Sars-Cov-2-Human PPI map,https://www.biorxiv.org/content/10.1101/2020.0...
147,RLAWWYSOJDYHDC-BZSNNMDCSA-N,Lisinopril,0,-2,Drugs from Sars-Cov-2-Human PPI map,https://www.biorxiv.org/content/10.1101/2020.0...
148,WIIZWVCIJKGZOK-RKDXNWHRSA-N,Chloramphenicol,0,-2,Drugs from Sars-Cov-2-Human PPI map,https://www.biorxiv.org/content/10.1101/2020.0...
149,CGZOGNUFXMNYEI-UDUHKTKDSA-N,Tigecycline,0,-2,Drugs from Sars-Cov-2-Human PPI map,https://www.biorxiv.org/content/10.1101/2020.0...


As you can see, some candidates were lost because they either didn't have a correct SMILES string or we couldn't find them in the CC universe of 800k molecules. Anyway, 151 rows is a lot!

## Ranking candidates

We have run the pipeline with:

* CC: Chemical Checker
* FP: Morgan Fingeprints (classical)

We compute matrices for:

* All evidence cutoffs
* All moas

Columns are:

* inchikey: candidate inchikey
* is_drug: yes (1), no (0)
* evidence: -2 (no evidence, most of the molecules), -1, 0, 1, 2, 3 (see spreadsheet)
* moa: -2 (no evidence, most of the molecules), 0, 1, 2, 3, 4, 5 (see spreadsheet)

Number of COVID molecules in:

* top_5: top 5 similarity
* lpv_5: -log p-value 5 (1e-5)
* lpv_4: -log p-value 4 (1e-4)
* lpv_3: -log p-value 3 (1e-3)
* lpv_2: -log p-value 2 (1e-2)

Finally:
* best_inchikey: COVID-19 molecule with the highest similarity to the candidate

In [55]:
with h5py.File("../web/data/dist_cc.h5", "r") as hf:
    r = hf["ranks_raw"][:]

In [58]:
np.min(r)

-1

In [ ]:
def load_df(simtype, evidence, moa, sort_by):
    if evidence is None or evidence < 0:
        evi_suf = "eviall"
    else:
        evi_suf = "evi%d" % evidence
    if moa is None:
        moa_suf = "moaall"
    else:
        moa_suf = "moa%d" % moa
    fn = "../outputs/df_cand_%s_%s_%s.pkl" % (simtype, evi_suf, moa_suf)
    df = pickle.load(open(fn, "rb"))
    return df.sort_values(sort_by, ascending=False)

Based on FP similarity, considering *all* candidates and *all* moas, and focusing on a p-value of 1e-3.

In [ ]:
load_df(simtype="fp", evidence=None, moa=None, sort_by="lpv_3")

Based on CC similarities, considering only evidence >= 1 and moa = 3 (p-value of 1e-4)

In [ ]:
load_df(simtype="cc", evidence=1, moa=3, sort_by="lpv_4")

In [ ]:
def counter(df):
    a = len(df[df["lpv_4"] > 0])
    df = df[df["is_drug"] == 1]
    b = len(df[df["lpv_4"] > 0])
    print("  Chem: %d" % a)
    print("  Drug: %d" % b)

for evid in [None, 0, 1, 2, 3]:
    for moa in [None, 0, 1, 2, 3, 4, 5]:
        print("Evidence: %s - MoA: %s" % (str(evid), str(moa)))
        df = load_df(simtype="cc", evidence=evid, moa=moa, sort_by="lpv_4")
        counter(df)
    

In [ ]:
x = np.array([[1,2,3,4],[1,2,3,4], [1,2,3,4]])

In [ ]:
x*[0,1,2,3]

## Manuscript plots

## Literature table

In [ ]:
evidence_legend = {
    -1: "Failed in clinics",
    0 : "Computational",
    1 : "Preclinical",
    2 : "Clinics",
    3 : "Clinics COVID19"
}

moa_legend = {
    -2: "NA",
    0 : "Unknown",
    1 : "Host factor",
    2 : "Virus entry",
    3 : "Protease inh.",
    4 : "RNA trans./rep.",
    5 : "Immunomodulator"
}

In [ ]:
def barplot(ax, df, column):
    counts = collections.defaultdict(int)
    for r in df[column].values:
        counts[r] += 1
    keys = sorted(counts.keys())
    vals = [counts[k] for k in keys]
    if column == "moa":
        keys = [-1, 0, 1, 2, 3, 4, 5]
    ax.scatter(vals, keys, c=keys, cmap="Spectral", edgecolor="black", s=80)
    if column == "moa":
        d = moa_legend
        yticklabels = [d[x] for x in sorted(d.keys())]
        yticks = [-1, 0, 1, 2, 3, 4, 5]
        ax.set_yticks(yticks)
        ax.set_yticklabels(yticklabels)
    else:
        d = evidence_legend
        yticks = sorted(d.keys())
        ax.set_yticks(yticks)
        ax.set_yticklabels([d[x] for x in yticks])
    if column == "moa":
        ax.set_title("MoA")
    else:
        ax.set_title(column.capitalize())
    ax.set_xlabel("Molecules")

fig, axs = plt.subplots(1,2, figsize=(8,2.2))
axs = axs.flatten()

barplot(axs[0], df_lit, "evidence")
barplot(axs[1], df_lit, "moa")
plt.tight_layout()

In [ ]:
M = np.zeros((7, 5))
for i, moa in enumerate([-2, 0, 1, 2, 3, 4, 5]):
    for j, evi in enumerate([-1, 0, 1, 2, 3]):
        n = len(df_lit[(df_lit["evidence"] == evi) & (df_lit["moa"] == moa)])
        d = len(df_lit[(df_lit["evidence"] == evi) | (df_lit["moa"] == moa)])
        M[i,j] = n/d

In [ ]:
plt.imshow(M)

## Similarity matrix

Now I explore the similarity matrix

In [ ]:
with h5py.File("../web/data/dist_trim_cc.h5", "r") as hf:
    ranks = hf["ranks"][:]
    print(hf.keys())

In [ ]:
ranks.shape

In [ ]:
idxs_i = np.argsort(-np.sum(ranks, axis=1))
idxs_j = np.argsort(-np.sum(ranks, axis=0))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,3))

ax.imshow(ranks[idxs_i][:,idxs_j], aspect="auto", cmap="GnBu")

In [ ]:
rr = ranks[idxs_i][:,idxs_j]

In [ ]:
set(rr.ravel())

In [ ]:
np.sum(rr, axis=1)